In [1]:
import pandas as pd

In [2]:
dataset=pd.read_csv("Social_Network_Ads.csv")

In [3]:
dataset

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0
...,...,...,...,...,...
395,15691863,Female,46,41000,1
396,15706071,Male,51,23000,1
397,15654296,Female,50,20000,1
398,15755018,Male,36,33000,0


In [4]:
dataset=pd.get_dummies(dataset,drop_first=True)

In [5]:
dataset

,User ID,Age,EstimatedSalary,Purchased,Gender_Male
0,15624510,19,19000,0,1
1,15810944,35,20000,0,1
2,15668575,26,43000,0,0
3,15603246,27,57000,0,0
4,15804002,19,76000,0,1
...,...,...,...,...,...
395,15691863,46,41000,1,0
396,15706071,51,23000,1,1
397,15654296,50,20000,1,0
398,15755018,36,33000,0,1


In [6]:
dataset["Purchased"].value_counts()
#refer how many class classified
#to find balanced or imbalanced
#unique entry &counts---->(0-143,1-143)->balanced
#(0-257,1-143)->imbalanced

0    257
1    143
Name: Purchased, dtype: int64

In [7]:
indep=dataset[["Age","EstimatedSalary","Gender_Male"]]
dep=dataset["Purchased"]

In [8]:
indep.shape#400-length ,3-i/p column

(400, 3)

In [9]:
dep

0      0
1      0
2      0
3      0
4      0
      ..
395    1
396    1
397    1
398    0
399    1
Name: Purchased, Length: 400, dtype: int64

In [10]:
#split into training & test set
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(indep,dep,test_size=0.30,random_state=0)



In [11]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
x_train=sc.fit_transform(x_train)
x_test=sc.transform(x_test)

In [12]:
#replace regression to classifier for classification
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
param_grid={'criterion':['gini','entropy'],'max_features': ['auto','sqrt','log2'],'n_estimators':[10,100]}
grid=GridSearchCV(RandomForestClassifier(),param_grid,refit=True,verbose=3,n_jobs=1,scoring='f1_weighted')
#f1_weighted suitable for all balanced &imbalance including precision,recall)
#fitting model for grid search
grid.fit(x_train,y_train)


Fitting 5 folds for each of 12 candidates, totalling 60 fits
[CV 1/5] END criterion=gini, max_features=auto, n_estimators=10;, score=0.874 total time=   0.0s
[CV 2/5] END criterion=gini, max_features=auto, n_estimators=10;, score=0.819 total time=   0.0s
[CV 3/5] END criterion=gini, max_features=auto, n_estimators=10;, score=0.823 total time=   0.0s
[CV 4/5] END criterion=gini, max_features=auto, n_estimators=10;, score=0.929 total time=   0.0s
[CV 5/5] END criterion=gini, max_features=auto, n_estimators=10;, score=0.890 total time=   0.0s
[CV 1/5] END criterion=gini, max_features=auto, n_estimators=100;, score=0.874 total time=   0.0s
[CV 2/5] END criterion=gini, max_features=auto, n_estimators=100;, score=0.840 total time=   0.0s
[CV 3/5] END criterion=gini, max_features=auto, n_estimators=100;, score=0.841 total time=   0.0s
[CV 4/5] END criterion=gini, max_features=auto, n_estimators=100;, score=0.929 total time=   0.0s
[CV 5/5] END criterion=gini, max_features=auto, n_estimators=1

GridSearchCV(estimator=RandomForestClassifier(), n_jobs=1,
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_features': ['auto', 'sqrt', 'log2'],
                         'n_estimators': [10, 100]},
             scoring='f1_weighted', verbose=3)

In [13]:
#print best parameter after tuning
#print(grid.best_params_)

re = grid.cv_results_

# Making predictions on the test data using the best estimator found by GridSearchCV



In [14]:

grid_predictions=grid.predict(x_test)

In [15]:
from sklearn.metrics import f1_score
f1_macro=f1_score(y_test,grid_predictions,average='weighted')
print("f1_macro value for best parameter{}:".format(grid.best_params_),f1_macro)#optional

f1_macro value for best parameter{'criterion': 'gini', 'max_features': 'sqrt', 'n_estimators': 100}: 0.9085936101092268


In [16]:
from sklearn.metrics import confusion_matrix
cm=confusion_matrix(y_test,grid_predictions)

In [17]:
# 79,0 correct diagonal,41,0 wrong diagonal
#type1 error should low
print(cm)

[[73  6]
 [ 5 36]]


In [18]:
from sklearn.metrics import classification_report
clf_report=classification_report(y_test,grid_predictions)

In [19]:
#overall performance(accuracy)=0.89
print(clf_report)

              precision    recall  f1-score   support

           0       0.94      0.92      0.93        79
           1       0.86      0.88      0.87        41

    accuracy                           0.91       120
   macro avg       0.90      0.90      0.90       120
weighted avg       0.91      0.91      0.91       120



In [20]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test,grid.predict_proba(x_test)[:,1])#receiver operating chareteristics area under curve(roc_auc)

0.9646495832046929

In [21]:
table = pd.DataFrame.from_dict(re)

In [22]:
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_criterion,param_max_features,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.008782,0.001972,0.002976,0.003661,gini,auto,10,"{'criterion': 'gini', 'max_features': 'auto', ...",0.874254,0.819142,0.823129,0.928571,0.890114,0.867042,0.041456,9
1,0.085866,0.016701,0.010417,0.006018,gini,auto,100,"{'criterion': 'gini', 'max_features': 'auto', ...",0.874254,0.840114,0.841398,0.929144,0.946153,0.886213,0.044074,4
2,0.009949,0.004959,0.000106,0.000212,gini,sqrt,10,"{'criterion': 'gini', 'max_features': 'sqrt', ...",0.855314,0.819142,0.824293,0.911105,0.870721,0.856115,0.033519,11
3,0.091023,0.013088,0.013559,0.002856,gini,sqrt,100,"{'criterion': 'gini', 'max_features': 'sqrt', ...",0.874254,0.840114,0.859435,0.911105,0.964286,0.889839,0.043896,1
4,0.008055,0.004428,0.004850,0.005101,gini,log2,10,"{'criterion': 'gini', 'max_features': 'log2', ...",0.835985,0.795918,0.859435,0.876368,0.909115,0.855364,0.038095,12
5,0.072326,0.006021,0.014251,0.005859,gini,log2,100,"{'criterion': 'gini', 'max_features': 'log2', ...",0.892857,0.821429,0.841398,0.911105,0.927778,0.878913,0.040818,5
6,0.008118,0.004996,0.003744,0.003571,entropy,auto,10,"{'criterion': 'entropy', 'max_features': 'auto...",0.874254,0.840114,0.841398,0.859025,0.907658,0.864490,0.024964,10
7,0.086917,0.009157,0.014319,0.004862,entropy,auto,100,"{'criterion': 'entropy', 'max_features': 'auto...",0.874254,0.857143,0.823129,0.911105,0.926743,0.878475,0.037230,6
8,0.007952,0.004629,0.004084,0.003748,entropy,sqrt,10,"{'criterion': 'entropy', 'max_features': 'sqrt...",0.874254,0.799537,0.821429,0.892857,0.963889,0.870393,0.057757,8
9,0.084560,0.009008,0.012538,0.003067,entropy,sqrt,100,"{'criterion': 'entropy', 'max_features': 'sqrt...",0.874254,0.875644,0.859435,0.911105,0.927778,0.889643,0.025535,2


In [23]:
age_input=int(input("Age:"))
estimated_salary=int(input("salary:"))
gender_male_input=int(input("sex male 0 or 1:"))


Age:21
salary:12222
sex male 0 or 1:1


In [24]:
Future_Prediction=grid.predict([[age_input,estimated_salary,gender_male_input]])
print("Future_Prediction{}".format(Future_Prediction))


Future_Prediction[1]


In [25]:
import pickle
filename="finalized_model_Grid.sav"
pickle.dump(grid,open(filename,'wb'))

In [26]:
loaded_model=pickle.load(open("finalized_model_Grid.sav",'rb'))
result=loaded_model.predict([[36,33000,0]])

In [28]:
 result

array([1], dtype=int64)